In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
raw_data = pd.read_csv("CompleteDataSet_training_competition_1500.csv", sep=",")

In [3]:
data = raw_data[['BeltAccelerometer x-axis', 'BeltAccelerometer y-axis', 'BeltAccelerometer z-axis', 'Activity']]

In [4]:
data

,BeltAccelerometer x-axis,BeltAccelerometer y-axis,BeltAccelerometer z-axis,Activity
0,0.146,0.895,0.367,1
1,0.146,0.895,0.367,1
2,0.146,0.895,0.367,1
3,0.146,0.895,0.367,1
4,0.178,0.896,0.373,1
5,0.160,0.895,0.372,1
6,0.160,0.895,0.372,1
7,0.160,0.895,0.372,1
8,0.160,0.895,0.372,1
9,0.160,0.895,0.372,1


In [5]:
n_step = 1500
n_batch = int(data.shape[0] / n_step)

# variables used in the paper
k = 15
t = 3
K = 300

xyz = 3

In [6]:
data = data.as_matrix()
calculate = np.zeros([n_batch, 12 * int(k / t) + 1])

# for each data
for i in range(n_batch):
    calculate[i][12 * int(k / t)] = data[i * n_step][xyz]
    
    r1 = np.zeros([xyz])
    r2 = np.zeros([xyz])
    
    # for each sliding window
    for j in range(int(k / t)):
        l1 = np.zeros([xyz])
        s1 = np.zeros([xyz])
        for m in range(xyz):
            l1[m] = data[i * n_step + j * K][m]
            s1[m] = pow(data[i * n_step + j * K][m], 2)
        
        l = np.zeros([xyz])
        s = np.zeros([xyz])
        for m in range(K):
            for n in range(xyz):
                l[n] += data[i * n_step + j * K + m][n]
                s[n] += pow(data[i * n_step + j * K + m][n], 2)
        
        for m in range(xyz):
            r1[m] += (l[m] - l1[m])
            r2[m] += (s[m] - s1[m])
        
            if not (j == 0):
                calculate[i][12 * j + 4 * m] = calculate[i][12 * (j - 1) + 4 * m + 2]
                calculate[i][12 * j + 4 * m + 1] = calculate[i][12 * (j - 1) + 4 * m + 3]
            calculate[i][12 * j + 4 * m + 2] = 2 * r1[m] / K
            if (r2[m] - (2 * pow(r1[m], 2) / K)) >= 0:
                calculate[i][12 * j + 4 * m + 3] = math.sqrt((r2[m] - (2 * pow(r1[m], 2) / K)) / (K / 2 - 1))
            else:
                calculate[i][12 * j + 4 * m + 3] = 0.0

In [7]:
calculate

array([[ 0.        ,  0.        ,  0.07258   , ..., -6.70912   ,
         0.        ,  1.        ],
       [ 0.        ,  0.        , -0.12129333, ..., -7.54506667,
         0.        ,  2.        ],
       [ 0.        ,  0.        , -0.35802667, ...,  8.54253333,
         0.        ,  3.        ],
       ...,
       [ 0.        ,  0.        , -0.04259333, ..., -7.0967    ,
         0.        ,  9.        ],
       [ 0.        ,  0.        , -0.10884   , ..., -7.10677333,
         0.        , 10.        ],
       [ 0.        ,  0.        ,  0.29968   , ...,  7.25137333,
         0.        , 11.        ]])

In [8]:
data = pd.DataFrame(calculate)

In [9]:
data.to_csv("training_data_SVM.csv", sep=",", header=False, index=False)